<a href="https://colab.research.google.com/github/nekoniii3/GPT_function_calling/blob/main/function_calling_%E3%82%A2%E3%83%8A%E3%83%AA%E3%82%B9%E3%83%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# OpenAIパッケージのインストール
!pip install openai

In [ ]:
import openai

openai.api_key = "ここをOpenAIキーに入れ替えてください。" #@param {type:"string"}


In [ ]:
import pandas_datareader.data as web

# DataReaderでstooqから株価情報を取得する関数
def get_stock_price(code, date):

  # 日本用のコードに変換
  code = code + '.JP'

  # DataReaderを利用しstooqからデータ取得
  df = web.DataReader(code, data_source='stooq', start=date,end=date)

  return df.to_json(orient='records')


# Chat-GPTに伝えるための関数の仕様を記したのJSON
stock_function =  {
    "name": "get_stock_price",
      "description": "証券コードと日付を引数として指定の会社のその日付の株価の情報を返します",
      "parameters": {
          "type": "object",
          "properties": {
              "code": {
                  "type": "string",
                  "description": "証券コード",
              },
              "date": {
                  "type": "string",
                  "description": "株価を知りたい日付",
              },
          },
          "required": ["code", "date"],
      },
}


In [ ]:
# GPTへ1回目の問い合わせ ※証券コードをたまに間違えるので、確実にするには指定してください
prompt = 'カルビー株式会社の2023年6月1日の株価の動きについて教えてください。'

response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=[{"role": "system", "content": "You are a capable securities analyst."}
     ,{"role": "user", "content": prompt}],
    functions=[stock_function],
    function_call="auto"
)

# GPTから1回目の返答
message = response["choices"][0]["message"]

# テスト出力
print(message)


In [ ]:
import json

# 回答にfunction_callの指示があった場合
if message.get("function_call"):

  # GPTの回答からget_stock_priceを実行
  function_name = message["function_call"]["name"]
  arguments=json.loads(message["function_call"]["arguments"])
  function_response = get_stock_price(
      code=arguments.get("code"),
      date=arguments.get("date")
  )

  # GPTへ2回目の問い合わせ
  response2 = openai.ChatCompletion.create(
      model="gpt-3.5-turbo-0613",
      messages=[{"role": "system", "content": "You are a capable securities analyst."}
          ,{"role": "user", "content": prompt},
          message,
          {
              "role": "function",
              "name": function_name,
              "content": function_response,
          },
      ],
  )

  # GPTからの回答（contentのみ）
  print(response2.choices[0]["message"]["content"])